# University Course Timetabling with Simulated Annealing

This notebook demonstrates the use of **Simulated Annealing (SA)** for solving the university course timetabling problem - a complex constraint satisfaction problem (CSP).

## What is Simulated Annealing?

Simulated Annealing is a probabilistic optimization technique inspired by the annealing process in metallurgy. It's particularly effective for:
- Large combinatorial optimization problems
- Problems with many local optima
- Constraint satisfaction problems

## Key Features of this Implementation

- **Multi-phase optimization**: First eliminate hard constraint violations, then optimize soft constraints
- **Tabu Search**: Prevents cycling back to recently visited solutions
- **Intensification phase**: Targeted search for remaining hard violations
- **Reheating**: Periodically increases temperature to escape local optima
- **Multiple move operators**: Diversified search strategies

## Problem Constraints

### Hard Constraints (must be satisfied)
- No room conflicts (one class per room at a time)
- No lecturer conflicts (one lecturer teaching at a time)
- No class conflicts (no overlapping classes for same program)
- Room capacity must accommodate class size
- Friday prayer time restrictions
- Class type time matching (morning classes in morning slots)

### Soft Constraints (optimization goals)
- Schedule compactness (minimize gaps)
- Preferred room/time assignments
- Research day preferences
- Minimize prayer time overlaps
- Proper room type usage

## 1. Setup and Imports

First, we install the package in development mode and import all necessary modules.

In [14]:
# Install package in development mode
%pip install pip install git+https://github.com/albertabayor/timetable-sa.git@feature/python-port#egg=timetable-sa&subdirectory=python


# Import standard libraries
import sys
import copy
import json
import random
import time as time_python

# Import pandas for data display
import pandas as pd

# Core framework
from timetable_sa import SimulatedAnnealing, SAConfig
from timetable_sa.core.interfaces.config import LoggingConfig

# Domain types
from timetable_sa.examples.timetabling.domain_types.domain import (
    Room, Lecturer, TimeSlot, ClassRequirement
)
from timetable_sa.examples.timetabling.domain_types.state import (
    ScheduleEntry, TimetableState
)

# All constraints and moves
from timetable_sa.examples.timetabling import (
    # Hard constraints
    NoRoomConflict, NoLecturerConflict, NoProdiConflict, RoomCapacity,
    MaxDailyPeriods, FridayTimeRestriction, NoFridayPrayConflict,
    PrayerTimeStart, ClassTypeTime, SaturdayRestriction,
    # Soft constraints
    Compactness, OverflowPenalty, PreferredRoom, PreferredTime,
    TransitTime, ResearchDay, PrayerTimeOverlap, EveningClassPriority,
    # Move generators
    ChangeTimeSlot, ChangeRoom, SwapClasses, ChangeTimeSlotAndRoom,
    FixRoomConflict, FixLecturerConflict, FixRoomCapacity,
    FixMaxDailyPeriods, FixFridayPrayerConflict, FixClassTypeTime,
    SwapFridayWithNonFriday,
)

# Utilities
from timetable_sa.examples.timetabling.utils.time import (
    time_to_minutes, calculate_end_time,
)
from timetable_sa.examples.timetabling.data.excel_loader import load_uisi_data

print("✅ All imports successful!")

Note: you may need to restart the kernel to use updated packages.
✅ All imports successful!


## 2. Configuration Parameters

Adjust these parameters to customize the optimization behavior.

In [25]:
# Data file path
EXCEL_DATA_PATH = "/home/emmanuelabayor/projects/timetable-sa/data_uisi.xlsx"

# SA Parameters
INITIAL_TEMPERATURE = 100000.0
MIN_TEMPERATURE = 0.0000001
COOLING_RATE = 0.9995
MAX_ITERATIONS = 100000

# Tabu Search (prevents cycling)
TABU_SEARCH_ENABLED = True
TABU_TENURE = 500  # Number of iterations to remember a state

# Reheating (helps escape local optima)
REHEATING_THRESHOLD = 2000  # Iterations without improvement before reheating
REHEATING_FACTOR = 2.0  # Temperature multiplier when reheating
MAX_REHEATS = 3  # Maximum number of reheating events

# Intensification (targeted search for hard violations)
ENABLE_INTENSIFICATION = True
INTENSIFICATION_ITERATIONS = 2000
MAX_INTENSIFICATION_ATTEMPTS = 3

# Set random seed for reproducibility
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

print("⚙️  Configuration loaded!")
print(f"   Temperature: {INITIAL_TEMPERATURE} → {MIN_TEMPERATURE}")
print(f"   Cooling rate: {COOLING_RATE}")
print(f"   Max iterations: {MAX_ITERATIONS}")
print(f"   Tabu search: {TABU_SEARCH_ENABLED} (tenure={TABU_TENURE})")

⚙️  Configuration loaded!
   Temperature: 100000.0 → 1e-07
   Cooling rate: 0.9995
   Max iterations: 100000
   Tabu search: True (tenure=500)


## 3. Load Data from Excel

Load the university data including rooms, lecturers, and class requirements.

In [26]:
print("=" * 60)
print("University Course Timetabling - Jupyter Notebook")
print("=" * 60)

# Load data
rooms, lecturers, classes = load_uisi_data(EXCEL_DATA_PATH)

# Display summary
print(f"\n📊 Data Summary:")
print(f"   Rooms: {len(rooms)}")
print(f"   Lecturers: {len(lecturers)}")
print(f"   Classes: {len(classes)}")

# Display sample data
print(f"\n🏠 Sample Rooms (first 5):")
df_rooms = pd.DataFrame([{
    'Code': r.code, 'Name': r.name, 'Type': r.type, 'Capacity': r.capacity
} for r in rooms[:5]])
display(df_rooms.style.hide(axis='index'))

print(f"\n👨‍🏫 Sample Lecturers (first 5):")
df_lecturers = pd.DataFrame([{
    'Code': l.code, 'Name': l.name, 'Prodi': l.prodi
} for l in lecturers[:5]])
display(df_lecturers.style.hide(axis='index'))

print(f"\n📚 Sample Classes (first 5):")
df_classes = pd.DataFrame([{
    'Code': c.kode_matakuliah, 'Name': c.mata_kuliah, 'Class': c.kelas,
    'SKS': c.sks, 'Participants': c.peserta
} for c in classes[:5]])
display(df_classes.style.hide(axis='index'))

University Course Timetabling - Jupyter Notebook

📊 Data Summary:
   Rooms: 33
   Lecturers: 99
   Classes: 373

🏠 Sample Rooms (first 5):


Code,Name,Type,Capacity
B2-R1,Kampus B B2 Ruang 1,theory,30
B3-R1,Kampus B B3 Ruang 1,theory,30
B3-R2,Kampus B B3 Ruang 2,theory,30
B3-R3,Kampus B B3 Ruang 3,theory,50
CM-101,Kampus B Gedung 1 Lantai 1 Ruang 1,theory,45



👨‍🏫 Sample Lecturers (first 5):


Code,Name,Prodi
RPA,"Dr. Rr. Rooswanti Putri Adi Agustini, S.Kom., M.M",Magister Management
GTK,"Dr. Ir. Gatot Kustyadji, S.E., M.Si., IPU., ASEAN Eng., APEC Eng.",Magister Management
ALF,"Dr. Alfina, S.M., M.M.",Magister Management
BAT,"Dr. Bambang Tutuko, S.E., M.M., CFP®",Magister Management
ANW,"Aditya Narendra Wardhana, S.T., M.SM.",Management



📚 Sample Classes (first 5):


Code,Name,Class,SKS,Participants
MM23EB03,Economic for Business,MM-1A,3,15
MM23SM03,Strategic Marketing Management,MM-1A,3,15
MM23HC03,Strategic Human Capital and Change Management,MM-1A,3,15
MM23CF03,Corporate Finance,MM-1A,3,15
MM23OS03,Operation and Supply Chain Management,MM-1A,3,15


## 4. Generate Time Slots

Create time slots for morning (PAGI) and evening sessions. The university has specific time periods:
- **Morning (PAGI)**: 07:30 - 15:30
- **Evening**: 18:00 - 21:00 (only evening slots from SORE period)

In [4]:
def generate_ts_slots(start_time: str, end_time: str, slot_duration: int = 50):
    """Generate time slots matching TypeScript implementation."""
    slots = []
    end_min = time_to_minutes(end_time)

    for day in ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]:
        hour = time_to_minutes(start_time) // 60
        minute = time_to_minutes(start_time) % 60
        period = 1

        while True:
            current_min = hour * 60 + minute
            if current_min >= end_min:
                break

            slot_start = f"{hour:02d}:{minute:02d}"
            end_min_calc = minute + slot_duration
            end_hour = hour + end_min_calc // 60
            end_minute = end_min_calc % 60

            end_time_calc = end_hour * 60 + end_minute
            if end_time_calc > end_min:
                break

            slot_end = f"{end_hour:02d}:{end_minute:02d}"

            if hour == 19 and minute == 20:
                slots.append(TimeSlot(day=day, start_time=slot_start,
                                     end_time=slot_end, period=period))
                break

            slots.append(TimeSlot(day=day, start_time=slot_start,
                                 end_time=slot_end, period=period))

            minute = end_minute
            hour_adjusted = False
            if minute == 50 and hour == 15:
                minute -= 20
            elif hour == 18 and minute == 50:
                minute -= 20
                hour_adjusted = True

            if minute >= 60:
                hour += minute // 60
                minute = minute % 60
            elif not hour_adjusted:
                hour = end_hour

            period += 1

    return slots

# Generate slots
pagi_slots = generate_ts_slots("07:30", "15:30", 50)
sore_slots = generate_ts_slots("15:30", "21:00", 50)
evening_slots = [s for s in sore_slots if time_to_minutes(s.start_time) >= time_to_minutes("18:00")]
time_slots = pagi_slots + evening_slots

print(f"\n⏰ Time Slots Generated:")
print(f"   Morning (PAGI): {len(pagi_slots)} slots")
print(f"   Evening: {len(evening_slots)} slots")
print(f"   Total: {len(time_slots)} slots")

# Show sample time slots
print(f"\n📅 Sample Monday Time Slots:")
monday_slots = [s for s in time_slots if s.day == 'Monday'][:10]
for slot in monday_slots:
    print(f"   Period {slot.period}: {slot.start_time} - {slot.end_time}")


⏰ Time Slots Generated:
   Morning (PAGI): 54 slots
   Evening: 18 slots
   Total: 72 slots

📅 Sample Monday Time Slots:
   Period 1: 07:30 - 08:20
   Period 2: 08:20 - 09:10
   Period 3: 09:10 - 10:00
   Period 4: 10:00 - 10:50
   Period 5: 10:50 - 11:40
   Period 6: 11:40 - 12:30
   Period 7: 12:30 - 13:20
   Period 8: 13:20 - 14:10
   Period 9: 14:10 - 15:00
   Period 4: 18:00 - 18:50


## 5. Create Initial State

Use a greedy algorithm to create the initial timetable. This algorithm:
1. Shuffles classes for random placement order
2. For each class, finds suitable rooms based on capacity and lab requirements
3. Checks for conflicts (room, lecturer, prodi/class)
4. Places classes without conflicts

In [13]:
print("\n🔨 Creating Initial State (Greedy Algorithm)...")

# We'll define the greedy initial state function directly in the notebook
# This makes the notebook self-contained and educational

def create_greedy_initial_state_v2(
    classes: list[ClassRequirement],
    rooms: list[Room],
    lecturers: list[Lecturer],
    time_slots: list[TimeSlot],
    random_seed: int | None = None
) -> TimetableState:
    """Create initial state using greedy assignment (matches TypeScript approach)."""
    if random_seed is not None:
        random.seed(random_seed)
    
    # Generate pagi and sore slots
    TIME_SLOTS_PAGI = generate_ts_slots("07:30", "15:30", 50)
    TIME_SLOTS_SORE = generate_ts_slots("15:30", "21:00", 50)
    EVENING_START_MINUTES = time_to_minutes("18:00")
    TIME_SLOTS_EVENING = [s for s in TIME_SLOTS_SORE if time_to_minutes(s.start_time) >= EVENING_START_MINUTES]
    
    schedule: list[ScheduleEntry] = []
    skipped = []
    success_count = 0
    
    def has_class_overlap(kelas1: str, kelas2: str) -> bool:
        """Check if two classes have overlapping class codes."""
        classes1 = [c.strip().upper() for c in kelas1.split(',')]
        classes2 = [c.strip().upper() for c in kelas2.split(',')]
        for c1 in classes1:
            for c2 in classes2:
                if c1 == c2:
                    return True
        return False
    
    def has_time_overlap(start1: str, end1: str, start2: str, end2: str) -> bool:
        """Check if two time ranges overlap."""
        s1 = time_to_minutes(start1)
        e1 = time_to_minutes(end1)
        s2 = time_to_minutes(start2)
        e2 = time_to_minutes(end2)
        return s1 < e2 and s2 < e1
    
    def has_conflict(new_entry: ScheduleEntry, existing_schedule: list[ScheduleEntry]) -> bool:
        """Check if new entry conflicts with any existing entry."""
        for existing in existing_schedule:
            if new_entry.time_slot.day != existing.time_slot.day:
                continue
            
            end1, _ = calculate_end_time(new_entry.time_slot.start_time, new_entry.sks, new_entry.time_slot.day)
            end2, _ = calculate_end_time(existing.time_slot.start_time, existing.sks, existing.time_slot.day)
            
            if not has_time_overlap(new_entry.time_slot.start_time, end1,
                                    existing.time_slot.start_time, end2):
                continue
            
            # Room conflict
            if new_entry.room == existing.room:
                return True
            
            # Lecturer conflict
            for lect in new_entry.lecturers:
                if lect in existing.lecturers:
                    return True
            
            # Prodi/Class conflict
            if new_entry.prodi == existing.prodi and has_class_overlap(new_entry.kelas, existing.kelas):
                return True
        
        return False
    
    # Shuffle classes for random placement order
    shuffled_classes = list(classes)
    random.shuffle(shuffled_classes)
    
    for class_req in shuffled_classes:
        lecturers_list = class_req.get_lecturers()
        
        if not lecturers_list:
            skipped.append(f"{class_req.kode_matakuliah}: No lecturers")
            continue
        
        # Get class properties
        participants = class_req.peserta
        needs_lab = class_req.needs_lab()
        class_type = class_req.class_type.lower()
        prodi = class_req.prodi
        sks = class_req.sks
        
        # Filter time slots by class type
        slots = TIME_SLOTS_SORE if class_type == 'sore' else TIME_SLOTS_PAGI
        
        # Filter Saturday for non-Magister Manajemen
        is_mm = 'magister manajemen' in prodi.lower()
        if not is_mm:
            slots = [s for s in slots if s.day != 'Saturday']
        
        if not slots:
            skipped.append(f"{class_req.kode_matakuliah}: No valid slots")
            continue
        
        placed = False
        
        for slot in slots:
            # Find suitable rooms
            suitable_rooms = []
            for r in rooms:
                if r.capacity < participants:
                    continue
                if needs_lab and 'lab' not in r.type.lower():
                    continue
                suitable_rooms.append(r)
            
            if not suitable_rooms:
                continue
            
            random.shuffle(suitable_rooms)
            
            for room in suitable_rooms:
                end_time, prayer_time = calculate_end_time(slot.start_time, sks, slot.day)
                
                entry = ScheduleEntry(
                    class_id=class_req.kode_matakuliah,
                    class_name=class_req.mata_kuliah,
                    kelas=class_req.kelas,
                    prodi=prodi,
                    lecturers=lecturers_list,
                    room=room.code,
                    time_slot=TimeSlot(
                        day=slot.day,
                        start_time=slot.start_time,
                        end_time=end_time,
                        period=slot.period
                    ),
                    sks=sks,
                    needs_lab=needs_lab,
                    participants=participants,
                    class_type=class_type,
                )
                
                if not has_conflict(entry, schedule):
                    schedule.append(entry)
                    placed = True
                    success_count += 1
                    break
            
            if placed:
                break
        
        if not placed:
            skipped.append(f"{class_req.kode_matakuliah}: No available slot")
    
    print(f"   Placed: {success_count}/{len(classes)}")
    if skipped:
        for s in skipped[:5]:
            print(f"   Skipped: {s}")
        if len(skipped) > 5:
            print(f"   ... and {len(skipped) - 5} more")
    
    return TimetableState(
        schedule=schedule,
        available_time_slots=time_slots,
        rooms=rooms,
        lecturers=lecturers,
    )

# Create the initial state
state = create_greedy_initial_state_v2(classes, rooms, lecturers, time_slots, RANDOM_SEED)

print(f"   Schedule entries: {len(state.schedule)}")
print(f"   Available rooms: {len(state.rooms)}")
print(f"   Available lecturers: {len(state.lecturers)}")

# Show sample schedule entries
print(f"\n📋 Sample Schedule Entries (first 5):")
for entry in state.schedule[:5]:
    print(f"   {entry.class_id}: {entry.class_name}")
    print(f"      {entry.time_slot.day} {entry.time_slot.start_time}-{entry.time_slot.end_time}")
    print(f"      Room: {entry.room}, Lecturers: {entry.lecturers}")


🔨 Creating Initial State (Greedy Algorithm)...
   Placed: 356/373
   Skipped: GS13TH46: No lecturers
   Skipped: GS13PW02: No lecturers
   Skipped: GS13IP12: No lecturers
   Skipped: CE11UT46: No lecturers
   Skipped: GS13PW02: No lecturers
   ... and 12 more
   Schedule entries: 356
   Available rooms: 33
   Available lecturers: 99

📋 Sample Schedule Entries (first 5):
   VD13GB03: Gambar Bentuk
      Monday 07:30-10:00
      Room: G4-R2, Lecturers: ['MNR']
   IS13ST03: Pengantar Sistem & Teknologi Informasi
      Monday 07:30-10:00
      Room: CM-203, Lecturers: ['TKN']
   ET13EE13: EKONOMI TEKNIK
      Monday 07:30-10:00
      Room: CM-201, Lecturers: ['LKT']
   CE11TH23: Termodinamika Teknik Kimia 1
      Monday 07:30-10:00
      Room: CM-207, Lecturers: ['YNK']
   LE12AJ53: Manajemen Resiko dan Kinerja
      Monday 10:00-12:30
      Room: G4-R1, Lecturers: ['LKT']


## 6. Define Constraints

Constraints define the rules that a valid timetable must follow.

In [14]:
# Hard constraints (MUST be satisfied)
hard_constraints = [
    NoRoomConflict(),           # No two classes in same room at same time
    NoLecturerConflict(),       # No lecturer teaching two classes simultaneously
    NoProdiConflict(),          # No overlapping classes for same program
    RoomCapacity(),             # Room capacity must accommodate class size
    MaxDailyPeriods(),          # Limit classes per day
    FridayTimeRestriction(),    # Specific time restrictions for Friday
    NoFridayPrayConflict(),     # Avoid prayer times on Friday
    PrayerTimeStart(),          # Start times must avoid prayer periods
    ClassTypeTime(),            # Morning classes in pagi slots, evening in sore
    SaturdayRestriction(),      # Non-MM programs can't use Saturday
]

# Soft constraints (optimization goals)
soft_constraints = [
    Compactness(),              # Minimize gaps in schedule
    OverflowPenalty(),          # Penalty for using lab rooms for non-lab classes
    PreferredRoom(),            # Preference for specific rooms
    PreferredTime(),            # Preference for specific times
    TransitTime(),              # Minimize time between classes
    ResearchDay(),              # Preferred research days for lecturers
    PrayerTimeOverlap(),        # Avoid prayer time overlaps
    EveningClassPriority(),     # Prioritize evening classes in sore slots
]

all_constraints = hard_constraints + soft_constraints

print(f"\n📋 Constraints:")
print(f"   Hard: {len(hard_constraints)}")
for c in hard_constraints:
    print(f"      - {c.name}")
print(f"   Soft: {len(soft_constraints)}")
for c in soft_constraints:
    print(f"      - {c.name}")


📋 Constraints:
   Hard: 10
      - No Room Conflict
      - No Lecturer Conflict
      - No Prodi Conflict
      - Room Capacity
      - Max Daily Periods
      - Friday Time Restriction
      - No Friday Prayer Conflict
      - Prayer Time Start
      - Class Type Time
      - Saturday Restriction
   Soft: 8
      - Compactness
      - Overflow Penalty
      - Preferred Room
      - Preferred Time
      - Transit Time
      - Research Day
      - Prayer Time Overlap
      - Evening Class Priority


## 7. Define Move Generators

Move generators are operators that modify the current solution to explore the search space.

In [15]:
move_generators = [
    ChangeTimeSlot(),          # Move class to different time
    ChangeRoom(),               # Move class to different room
    SwapClasses(),              # Exchange time slots between two classes
    ChangeTimeSlotAndRoom(),    # Change both time and room
    FixRoomConflict(),          # Specialized: Fix room conflicts
    FixLecturerConflict(),      # Specialized: Fix lecturer conflicts
    FixRoomCapacity(),          # Specialized: Fix capacity issues
    FixMaxDailyPeriods(),       # Specialized: Fix daily period violations
    FixFridayPrayerConflict(),  # Specialized: Fix Friday prayer conflicts
    FixClassTypeTime(),         # Specialized: Fix class type time mismatches
    SwapFridayWithNonFriday(),  # Specialized: Swap Friday/non-Friday classes
]

print(f"\n🔀 Move Generators: {len(move_generators)}")
for gen in move_generators:
    print(f"   - {gen.name}")


🔀 Move Generators: 11
   - Change Time Slot
   - Change Room
   - Swap Classes
   - Change Time Slot and Room
   - Fix Room Conflict
   - Fix Lecturer Conflict
   - Fix Room Capacity
   - Fix Max Daily Periods
   - Fix Friday Prayer Conflict
   - Fix Class Type Time
   - Swap Friday With Non Friday


## 8. Configure Simulated Annealing

Set up the Simulated Annealing algorithm with all parameters.

In [16]:
config = SAConfig(
    initial_temperature=INITIAL_TEMPERATURE,
    min_temperature=MIN_TEMPERATURE,
    cooling_rate=COOLING_RATE,
    max_iterations=MAX_ITERATIONS,
    hard_constraint_weight=10000.0,
    tabu_search_enabled=TABU_SEARCH_ENABLED,
    tabu_tenure=TABU_TENURE,
    max_tabu_list_size=1000,
    reheating_threshold=REHEATING_THRESHOLD,
    reheating_factor=REHEATING_FACTOR,
    max_reheats=MAX_REHEATS,
    enable_intensification=ENABLE_INTENSIFICATION,
    intensification_iterations=INTENSIFICATION_ITERATIONS,
    max_intensification_attempts=MAX_INTENSIFICATION_ATTEMPTS,
    clone_state=lambda s: TimetableState(
        schedule=copy.deepcopy(s.schedule),
        available_time_slots=s.available_time_slots,
        rooms=s.rooms,
        lecturers=s.lecturers,
    ),
    logging=LoggingConfig(
        enabled=True,
        level="info",
        log_interval=1000,
        output="console",
    ),
)

print(f"\n⚙️  SA Configuration:")
print(f"   Temperature: {config.initial_temperature} → {config.min_temperature}")
print(f"   Cooling rate: {config.cooling_rate}")
print(f"   Max iterations: {config.max_iterations}")
print(f"   Tabu search: {config.tabu_search_enabled} (tenure={config.tabu_tenure})")
print(f"   Intensification: {config.enable_intensification}")


⚙️  SA Configuration:
   Temperature: 100000.0 → 1e-07
   Cooling rate: 0.9995
   Max iterations: 100000
   Tabu search: True (tenure=500)
   Intensification: True


## 9. Run Optimization

Execute the Simulated Annealing algorithm. This may take several minutes depending on your MAX_ITERATIONS setting.

**Note**: The optimization will log progress every 1000 iterations. Watch for:
- Phase 1: Eliminating hard constraint violations
- Phase 1.5: Intensification (if hard violations remain)
- Phase 2: Optimizing soft constraints
- Reheating events (when stuck in local optima)

In [17]:
print("\n" + "=" * 60)
print("🚀 Running Optimization...")
print("=" * 60)

sa = SimulatedAnnealing(state, all_constraints, move_generators, config)

start_time = time_python.time()
result = sa.solve()
elapsed_time = time_python.time() - start_time

print("\n" + "=" * 60)
print("✅ Optimization Complete!")
print("=" * 60)


🚀 Running Optimization...
[2026-01-11T02:28:43.670363] [INFO] Simulated Annealing initialized {'hard_constraints': 10, 'soft_constraints': 8, 'move_generators': 11, 'config': {'initial_temperature': 100000.0, 'min_temperature': 1e-07, 'cooling_rate': 0.9995, 'max_iterations': 100000}}
[2026-01-11T02:28:43.670492] [INFO] Starting optimization...
[2026-01-11T02:28:43.670511] [INFO] Phase 1: Eliminating hard constraint violations
[2026-01-11T02:28:43.694877] [INFO] Initial state {'fitness': '18795.81', 'hard_violations': 8}
[2026-01-11T02:29:42.272232] [INFO] [Phase 1] Iteration 1000: Temp = 66291.88, Hard violations = 5, Best = 5
[2026-01-11T02:31:22.828307] [INFO] [Phase 1] Iteration 3000: Temp = 29499.25, Hard violations = 4, Best = 4
[2026-01-11T02:32:14.280223] [INFO] [Phase 1] Iteration 4000: Temp = 19129.98, Hard violations = 4, Best = 4
[2026-01-11T02:33:05.055459] [INFO] [Phase 1] Iteration 5000: Temp = 12567.97, Hard violations = 2, Best = 2
[2026-01-11T02:33:32.890067] [INFO] 

## 10. Display Results

Analyze the optimization results including fitness, violations, and operator statistics.

In [18]:
print(f"\n📊 Results:")
print(f"   Final fitness: {result['fitness']:.4f}")
print(f"   Hard violations: {result['hard_violations']}")
print(f"   Soft violations: {result['soft_violations']}")
print(f"   Iterations: {result['iterations']}")
print(f"   Reheats: {result.get('reheats', 0)}")
print(f"   Execution time: {elapsed_time:.2f} seconds ({elapsed_time/60:.2f} minutes)")

# Operator statistics
print(f"\n🔧 Operator Statistics:")
df_stats = pd.DataFrame(result['operator_stats']).T
df_stats.columns = ['Attempts', 'Improvements', 'Accepted', 'Success Rate']
df_stats['Success Rate'] = df_stats['Success Rate'].apply(lambda x: f"{x*100:.2f}%")
display(df_stats.sort_values('Attempts', ascending=False).style.hide(axis='index'))

# Constraint violations
print(f"\n📋 Constraint Violations:")
violations_summary = {}
for constraint in all_constraints:
    violations = constraint.get_violations(result['state'])
    if violations:
        violations_summary[constraint.name] = len(violations)

if violations_summary:
    df_violations = pd.DataFrame(list(violations_summary.items()),
                                  columns=['Constraint', 'Violations'])
    display(df_violations.sort_values('Violations', ascending=False).style.hide(axis='index'))
else:
    print("   No violations! Perfect schedule! 🎉")


📊 Results:
   Final fitness: 61.5674
   Hard violations: 0
   Soft violations: 129
   Iterations: 81348
   Reheats: 3
   Execution time: 4369.65 seconds (72.83 minutes)

🔧 Operator Statistics:


Attempts,Improvements,Accepted,Success Rate
18140.000000,1450.000000,8148.000000,7.99%
15717.000000,127.000000,4494.000000,0.81%
15486.000000,1012.000000,8080.000000,6.53%
11152.000000,6.000000,9912.000000,0.05%
1043.000000,57.000000,64.000000,5.47%
671.000000,8.000000,283.000000,1.19%
117.000000,11.000000,110.000000,9.40%
2.000000,2.000000,2.000000,100.00%
1.000000,1.000000,1.000000,100.00%
0.000000,0.000000,0.000000,0.00%



📋 Constraint Violations:


Constraint,Violations
Research Day,51
Preferred Time,22
Evening Class Priority,17
Compactness,16
Prayer Time Overlap,15
Preferred Room,6
Overflow Penalty,2


## 11. Save Results to JSON

Export the final schedule to a JSON file for further analysis or integration.

In [13]:
final_state = result['state']

# Create flattened version for display and nested version for export
schedule_result_flat = []
schedule_result_nested = []

for entry in final_state.schedule:
    end_time, prayer_time_added = calculate_end_time(
        entry.time_slot.start_time, entry.sks, entry.time_slot.day
    )
    is_overflow = not entry.needs_lab and 'lab' in entry.room.lower()

    # Flattened version for display
    schedule_result_flat.append({
        "classId": entry.class_id,
        "className": entry.class_name,
        "class": entry.kelas,
        "prodi": entry.prodi,
        "lecturers": entry.lecturers,
        "room": entry.room,
        "day": entry.time_slot.day,
        "period": entry.time_slot.period,
        "startTime": entry.time_slot.start_time,
        "endTime": entry.time_slot.end_time,
        "sks": entry.sks,
        "needsLab": entry.needs_lab,
        "participants": entry.participants,
        "classType": entry.class_type,
        "prayerTimeAdded": prayer_time_added,
        "isOverflowToLab": is_overflow,
    })
    
    # Nested version for JSON export (matches TypeScript format)
    schedule_result_nested.append({
        "classId": entry.class_id,
        "className": entry.class_name,
        "class": entry.kelas,
        "prodi": entry.prodi,
        "lecturers": entry.lecturers,
        "room": entry.room,
        "timeSlot": {
            "period": entry.time_slot.period,
            "day": entry.time_slot.day,
            "startTime": entry.time_slot.start_time,
            "endTime": entry.time_slot.end_time,
        },
        "sks": entry.sks,
        "needsLab": entry.needs_lab,
        "participants": entry.participants,
        "classType": entry.class_type,
        "prayerTimeAdded": prayer_time_added,
        "isOverflowToLab": is_overflow,
    })

final_result = {
    "fitness": result['fitness'],
    "hardViolations": result['hard_violations'],
    "softViolations": result['soft_violations'],
    "iterations": result['iterations'],
    "schedule": schedule_result_nested,
}

OUTPUT_PATH = "/home/emmanuelabayor/projects/timetable-sa/python/timetable-result.json"
with open(OUTPUT_PATH, 'w') as f:
    json.dump(final_result, f, indent=2, ensure_ascii=False)

print(f"\n💾 Results saved to: {OUTPUT_PATH}")
print(f"   Schedule entries: {len(schedule_result_nested)}")

NameError: name 'result' is not defined

## 12. Sample Schedule Display

Display the generated schedule in a readable format.

In [12]:
# Create DataFrame from schedule (using flattened version from previous cell)
df_schedule = pd.DataFrame(schedule_result_flat)

# Display first 10 entries
print(f"\n📅 Sample Schedule (first 10 entries):")
display_cols = ['classId', 'className', 'class', 'day', 'startTime', 'endTime', 'room', 'lecturers']
df_display = df_schedule[display_cols].head(10)
display(df_display.style.hide(axis='index'))

# Display schedule grouped by day
print(f"\n📅 Schedule by Day:")
for day in ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]:
    day_schedule = df_schedule[df_schedule['day'] == day].sort_values('startTime')
    if len(day_schedule) > 0:
        print(f"\n### {day} ({len(day_schedule)} classes)")
        display_cols = ['startTime', 'endTime', 'classId', 'className', 'room']
        display(day_schedule[display_cols].head(10).style.hide(axis='index'))

NameError: name 'schedule_result_flat' is not defined

## Summary

You've successfully run the Simulated Annealing optimization for university course timetabling!

### Key Takeaways:

1. **Multi-phase Optimization**: The algorithm first focuses on eliminating hard constraint violations (critical for feasibility), then optimizes soft constraints (quality improvements).

2. **Tabu Search**: By remembering recently visited states, the algorithm avoids cycling and explores new areas of the solution space.

3. **Reheating**: When stuck in a local optimum, the algorithm temporarily increases temperature to allow more exploration.

4. **Move Operators**: Different move operators (change time, change room, swap, etc.) provide diverse ways to explore the solution space.

### Next Steps:

- Experiment with different parameter values (temperature, cooling rate, tabu tenure)
- Add visualization cells to analyze room utilization and lecturer workload
- Compare results with different random seeds
- Integrate the schedule into a university management system